In [1]:
import re
import json
import random

# Load

In [2]:
def read_json_file(file_path):
    with open(file_path, 'r', encoding="utf-8") as file:
        json_data = json.load(file)
    return json_data
# ac = read_json_file('./alpaca_chat_data.json')
mc = read_json_file('./medical_chat_data.json')
qc = read_json_file('./quora_chat_data.json')
sc = read_json_file('./stackoverflow_chat_data.json')

# Sample

In [3]:
session_cnt = len(mc)+len(qc)+len(sc)
# ac_ratio = len(ac)/session_cnt
mc_ratio = len(mc)/session_cnt
qc_ratio = len(qc)/session_cnt
sc_ratio = len(sc)/session_cnt
# print(f'ac_ratio:{ac_ratio}')
print(f'mc_ratio:{mc_ratio}')
print(f'qc_ratio:{qc_ratio}')
print(f'sc_ratio:{sc_ratio}')

mc_ratio:0.29593544191098003
qc_ratio:0.3438551736766665
sc_ratio:0.36020938441235345


In [4]:
# num_ac_sample = round(10000*ac_ratio)+1
num_mc_sample = round(10000*mc_ratio)
num_qc_sample = round(10000*qc_ratio)
num_sc_sample = round(10000*sc_ratio)
# print(num_ac_sample)
print(num_mc_sample)
print(num_qc_sample) 
print(num_sc_sample)

2959
3439
3602


In [5]:
assert ((num_mc_sample+num_qc_sample+num_sc_sample) == 10000)

In [6]:
random_seed = 125
random.seed(random_seed)
# random.shuffle(ac)
random.shuffle(mc)
random.shuffle(qc)
random.shuffle(sc)

In [7]:
# check
mc[0]
# {'topic': 'What is (are) Pachygyria ?',
# 'input': 'The conversation between human and AI assistant.\n[|Human|] Can you

{'topic': 'What is (are) Pachygyria ?',
 'input': 'The conversation between human and AI assistant.\n[|Human|] Can you tell me what Pachygyria is?\n[|AI|] Pachygyria is a rare brain malformation where the folds in the cerebral cortex (the outer layer of the brain) are abnormally thick and reduced in number. This can cause developmental delays, seizures, and other neurological symptoms in affected individuals.\n[|Human|] What causes Pachygyria?\n[|AI|] Pachygyria can be caused by genetic mutations or other factors that disrupt the normal development of the brain. It can also be associated with other genetic syndromes.\n[|Human|] Is there a cure for Pachygyria?\n[|AI|] Unfortunately, there is no cure for Pachygyria at this time. Treatment focuses on managing symptoms and improving quality of life for affected individuals.\n[|Human|] What are some of the symptoms of Pachygyria?\n[|AI|] Some of the symptoms of Pachygyria include developmental delays, seizures, muscle weakness or stiffness,

In [8]:
# ac_10k = ac[:num_ac_sample]
mc_10k = mc[:num_mc_sample]
qc_10k = qc[:num_qc_sample]
sc_10k = sc[:num_sc_sample]

bz_10k = []
# bz_10k.extend(ac_10k)
bz_10k.extend(mc_10k)
bz_10k.extend(qc_10k)
bz_10k.extend(sc_10k)
assert len(bz_10k) == 10000

In [9]:
random_seed = 125
random.seed(random_seed)
random.shuffle(bz_10k)
print(bz_10k[0])
# {'topic': 'What are the symptoms of Subaortic stenosis short stature syndrome ?'

{'topic': 'What are the symptoms of Subaortic stenosis short stature syndrome ?', 'input': "The conversation between human and AI assistant.\n[|Human|] Can you tell me about the symptoms of Subaortic stenosis short stature syndrome?\n[|AI|] Sure, Subaortic stenosis short stature syndrome is a rare disorder. The symptoms can include short stature, narrowing of the aortic valve, fatigue, exercise intolerance, among others.\n[|Human|] Is this disease genetic?\n[|AI|] Yes, it is usually inherited as an autosomal dominant trait, which means if one parent has the condition, there is a 50% chance that their child will inherit the gene and the condition.\n[|Human|] How is it diagnosed?\n[|AI|] A doctor may perform a physical exam and listen to the patient's heart for any abnormalities. An echocardiogram may also be done to determine the extent of the heart condition.\n[|Human|] Is there any treatment for this?\n[|AI|] Treatment options for Subaortic stenosis short stature syndrome may include 

In [10]:
bz_10k[5]

{'topic': 'What would happen to the amount submitted by me through challent after valid date of challan submission?',
 'input': 'The conversation between human and AI assistant.\n[|Human|] I have a question about the challan submission. What would happen to the amount submitted by me through the challan after the valid date of challan submission?\n[|AI|] If you have submitted the challan after the valid date, then the amount will not be accepted by the concerned authorities. It is always advisable to submit the challan before the due date to avoid any inconvenience.\n[|Human|] Is there any way to get the amount back or will it be forfeited?\n[|AI|] It depends on the concerned authorities. It is always better to contact the authority and ask if there is any way to get the amount back. However, it is highly unlikely that the amount will be refunded.\n[|Human|] Can I submit the challan again if I missed the valid date?\n[|AI|] No, you cannot submit the challan again if you missed the vali

# Remove

In [12]:
for session_dict in bz_10k:
    session_str = session_dict['input']
    session_str = re.sub(r'\[\|Human\|\].*$', '', session_str) 
    session_list = re.split(r'(\[\|Human\|\]|\[\|AI\|\] )', session_str)
    session_list = [i.strip() for i in session_list if i.strip()]
    session_list = session_list[1:]
    removed_list = []
    for i in session_list:
        if i == '[|Human|]' or i == '[|AI|]':
            continue
        removed_list.append(i)
    
    conv_list = []
    for i, utter in enumerate(removed_list):
        if i % 2 == 0: 
            conv_list.append({'from': 'human', 'value': utter})
        else: 
            conv_list.append({'from': 'gpt', 'value': utter})
    session_dict['conversations'] = conv_list

    del session_dict['input']

In [13]:
# check
bz_10k[0]

{'topic': 'What are the symptoms of Subaortic stenosis short stature syndrome ?',
 'conversations': [{'from': 'human',
   'value': 'Can you tell me about the symptoms of Subaortic stenosis short stature syndrome?'},
  {'from': 'gpt',
   'value': 'Sure, Subaortic stenosis short stature syndrome is a rare disorder. The symptoms can include short stature, narrowing of the aortic valve, fatigue, exercise intolerance, among others.'},
  {'from': 'human', 'value': 'Is this disease genetic?'},
  {'from': 'gpt',
   'value': 'Yes, it is usually inherited as an autosomal dominant trait, which means if one parent has the condition, there is a 50% chance that their child will inherit the gene and the condition.'},
  {'from': 'human', 'value': 'How is it diagnosed?'},
  {'from': 'gpt',
   'value': "A doctor may perform a physical exam and listen to the patient's heart for any abnormalities. An echocardiogram may also be done to determine the extent of the heart condition."},
  {'from': 'human', 'va

# Output

In [14]:
with open('baize_10k_for_train.json', 'w', encoding="utf-8" ) as file:
    json.dump(bz_10k, file, indent=2, ensure_ascii=False)